In [1]:
import os
import csv
import threading
import openpyxl
from openpyxl.utils import get_column_letter
import pandas as pd
import tkinter as tk
from tkinter import ttk, filedialog, messagebox, simpledialog

class Application(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Фильтрация организаций")
        self.geometry("750x500")
        
        # self.style = ttk.Style()
        # self.style.theme_use('clam')
        
        # Переменные
        self.input_file = tk.StringVar()
        self.output_folder = tk.StringVar()
        self.column_name = tk.StringVar()
        self.sheet_name = tk.StringVar()
        self.header_row = tk.IntVar(value=1)
        self.status_text = tk.StringVar()
        self.progress_value = tk.DoubleVar()
        self.prefix = tk.StringVar()
        self.columns_list = []
        self.sheet_names = []
        self.preview_data = []

        # Флаг для остановки потока
        self.running = False
        
        self.create_widgets()
        self.protocol("WM_DELETE_WINDOW", self.on_close)
    
    def create_widgets(self):
        input_frame = ttk.LabelFrame(self, text="Параметры обработки")
        input_frame.pack(padx=10, pady=5, fill='x')
        
        # Выбор файла
        ttk.Label(input_frame, text="Входной файл:").grid(row=0, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.input_file, width=40).grid(row=0, column=1, padx=5, pady=2)
        ttk.Button(input_frame, text="Обзор...", command=self.browse_file).grid(row=0, column=2, padx=5, pady=2)
        
        # Выбор листа
        self.sheet_label = ttk.Label(input_frame, text="Лист в книге:")
        self.sheet_label.grid(row=1, column=0, sticky='w', padx=5, pady=2)
        self.sheet_combobox = ttk.Combobox(input_frame, textvariable=self.sheet_name, state="readonly", width=20)
        self.sheet_combobox.grid(row=1, column=1, sticky='w', padx=5, pady=2)
        
        # Выбор строки заголовка
        ttk.Label(input_frame, text="Строка с заголовками:").grid(row=2, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.header_row, width=5).grid(row=2, column=1, sticky='w', padx=5, pady=2)
        ttk.Button(input_frame, text="Предпросмотр", command=self.show_preview).grid(row=2, column=2, padx=5, pady=2)
        
        # Выбор столбца
        ttk.Label(input_frame, text="Столбец с организациями:").grid(row=3, column=0, sticky='w', padx=5, pady=2)
        self.column_combobox = ttk.Combobox(input_frame, textvariable=self.column_name, state="readonly", width=20)
        self.column_combobox.grid(row=3, column=1, sticky='w', padx=5, pady=2)
        
        # Выбор папки
        ttk.Label(input_frame, text="Папка для результатов:").grid(row=4, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.output_folder, width=40).grid(row=4, column=1, padx=5, pady=2)
        ttk.Button(input_frame, text="Обзор...", command=self.browse_output).grid(row=4, column=2, padx=5, pady=2)

        # prefix
        ttk.Label(input_frame, text="Префикс для названия файлов:").grid(row=5, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.prefix, width=40).grid(row=5, column=1, padx=5, pady=2)
        self.prefix.set('Гепатиты 849-п август')
        
        # Прогресс и статус
        progress_frame = ttk.Frame(self)
        progress_frame.pack(padx=10, pady=10, fill='x')
        self.progress = ttk.Progressbar(progress_frame, variable=self.progress_value, maximum=100)
        self.progress.pack(fill='x')
        
        status_frame = ttk.Frame(self)
        status_frame.pack(padx=10, pady=5, fill='x')
        ttk.Label(status_frame, textvariable=self.status_text).pack(side='left')
        
        # Кнопки
        button_frame = ttk.Frame(self)
        button_frame.pack(padx=10, pady=10)
        ttk.Button(button_frame, text="Старт", command=self.start_processing).pack(side='left', padx=5)
        ttk.Button(button_frame, text="Выход", command=self.on_close).pack(side='left', padx=5)
        
    def browse_file(self):
        filetypes = (("Excel/CSV файлы", "*.xlsx *.xls *.csv"), ("Все файлы", "*.*"))
        filename = filedialog.askopenfilename(filetypes=filetypes)
        if filename:
            self.input_file.set(filename)
            self.load_file_metadata(filename)
    
    def load_file_metadata(self, filename):
        try:
            if filename.lower().endswith(('.xls', '.xlsx')):
                self.load_sheets(filename)
                self.load_columns()
            else:
                self.load_columns()
            
            self.show_preview()
            
        except Exception as e:
            messagebox.showerror("Ошибка", f"Ошибка загрузки файла: {str(e)}")
    
    def load_sheets(self, filename):
        try:
            wb = openpyxl.load_workbook(filename, read_only=True)
            self.sheet_names = wb.sheetnames
            wb.close()
            self.sheet_combobox['values'] = self.sheet_names
            self.sheet_name.set(self.sheet_names[0])
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось прочитать листы: {str(e)}")
    
    def load_columns(self):
        try:
            header_row = self.header_row.get() - 1  # Конвертируем в 0-based индекс
            filename = self.input_file.get()
            
            if filename.endswith(('.xls', '.xlsx')):
                df = pd.read_excel(
                    filename,
                    sheet_name=self.sheet_name.get(),
                    header=header_row,
                    nrows=0
                )
            else:
                df = pd.read_csv(
                    filename,
                    header=header_row,
                    nrows=0,
                    encoding='utf-8'
                )
            
            self.columns_list = df.columns.tolist()
            self.column_combobox['values'] = self.columns_list
            if self.columns_list:
                self.column_name.set(self.columns_list[0])
            
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось прочитать заголовки: {str(e)}")
    
    def show_preview(self):
        try:
            filename = self.input_file.get()
            if not filename:
                return
            
            # Чтение первых 15 строк для предпросмотра
            nrows = 15
            header_row = self.header_row.get() - 1
            
            if filename.endswith(('.xls', '.xlsx')):
                df = pd.read_excel(
                    filename,
                    sheet_name=self.sheet_name.get(),
                    header=None,
                    nrows=nrows
                )
            else:
                df = pd.read_csv(
                    filename,
                    header=None,
                    nrows=nrows,
                    encoding='utf-8'
                )
            
            # Создание окна предпросмотра
            preview_window = tk.Toplevel(self)
            preview_window.title("Предпросмотр данных")
            
            # Создаем Treeview с прокруткой
            frame = ttk.Frame(preview_window)
            frame.pack(expand=True, fill='both', padx=5, pady=5)
            
            tree = ttk.Treeview(frame, columns=list(range(len(df.columns))), show="headings")
            vsb = ttk.Scrollbar(frame, orient="vertical", command=tree.yview)
            hsb = ttk.Scrollbar(frame, orient="horizontal", command=tree.xview)
            tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)
            
            tree.grid(row=0, column=0, sticky='nsew')
            vsb.grid(row=0, column=1, sticky='ns')
            hsb.grid(row=1, column=0, sticky='ew')
            frame.grid_columnconfigure(0, weight=1)
            frame.grid_rowconfigure(0, weight=1)
            
            # Настройка столбцов
            for i in range(len(df.columns)):
                tree.heading(i, text=f"Col {i+1}")
                tree.column(i, width=100, anchor='w')
            
            # Вставка данных
            for index, row in df.iterrows():
                tree.insert("", "end", values=tuple(row))
            
            # Подсветка строки с заголовками
            if header_row < len(df):
                tree.tag_configure('header', background='#b3e6ff')
                children = tree.get_children()
                if header_row < len(children):
                    tree.item(children[header_row], tags=('header',))
            
            # Обработка двойного клика
            def on_double_click(event):
                # Получаем выбранную строку и столбец
                item = tree.identify_row(event.y)
                column = tree.identify_column(event.x)
                
                if item and column:
                    # Получаем индекс строки
                    children = tree.get_children()
                    row_index = children.index(item) + 1  # +1 так как нумерация с 1
                    
                    # Получаем индекс столбца
                    col_index = int(column.replace("#", "")) - 1  # Преобразуем "#1" в 0
                    
                    # Устанавливаем новую строку заголовков
                    self.header_row.set(row_index)
                    self.load_columns()
                    
                    # Устанавливаем выбранный столбец
                    if self.columns_list and col_index < len(self.columns_list):
                        self.column_name.set(self.columns_list[col_index])
                    
                    # Закрываем окно предпросмотра
                    preview_window.destroy()
            
            # Привязываем обработчик двойного клика
            tree.bind("<Double-1>", on_double_click)
            
            # Кнопка закрытия
            button_frame = ttk.Frame(preview_window)
            button_frame.pack(fill='x', padx=5, pady=5)
            ttk.Button(button_frame, text="Закрыть", command=preview_window.destroy).pack(side='right')
            
            # Установка размера окна
            preview_window.geometry("800x400")
            
            # Инструкция для пользователя
            instruction = ttk.Label(
                preview_window,
                text="Дважды щелкните по ячейке, чтобы выбрать строку заголовков и столбец для фильтрации",
                font=('Arial', 10)
            )
            instruction.pack(pady=5)
            
        except Exception as e:
            messagebox.showerror("Ошибка", f"Ошибка предпросмотра: {str(e)}")
    
    def browse_output(self):
        folder = filedialog.askdirectory()
        if folder:
            self.output_folder.set(folder)
    
    def start_processing(self):
        if not self.validate_input():
            return
        # Запуск обработки в отдельном потоке
        self.running = True
        self.processing_thread = threading.Thread(target=self.process_data)
        self.processing_thread.start()
    
    def validate_input(self):
        if not self.input_file.get():
            messagebox.showwarning("Ошибка", "Выберите входной файл!")
            return False
        if self.input_file.get().endswith(('.xls', '.xlsx')) and not self.sheet_name.get():
            messagebox.showwarning("Ошибка", "Выберите лист в книге!")
            return False
        if not self.column_name.get():
            messagebox.showwarning("Ошибка", "Выберите столбец с организациями!")
            return False
        return True
    
    def process_data(self):
        vipnets = {}
        with open('act_addr.txt') as in_file:
            csv_reader = csv.reader(in_file)
            for row in csv_reader:
                vipnets[row[0]]=row[1]
        try:
            self.after(0, self.update_status, "Начало обработки...")
            
            input_file = self.input_file.get()
            column_name = self.column_name.get()
            output_folder = self.output_folder.get() or 'output'
            sheet = self.sheet_name.get() if self.sheet_name.get() else None
            header_row = self.header_row.get() - 1
            
            # Загрузка данных
            if input_file.endswith('.csv'):
                df = pd.read_csv(input_file, header=header_row)
                col_widths = None
            else:
                wb = openpyxl.load_workbook(input_file)
                ws = wb[sheet] if sheet else wb.active
                
                # Чтение ширины столбцов
                col_widths = []
                for col in range(1, ws.max_column + 1):
                    col_letter = get_column_letter(col)
                    col_widths.append(
                        ws.column_dimensions[col_letter].width 
                        if col_letter in ws.column_dimensions 
                        else 8.43
                    )
                
                df = pd.read_excel(input_file, sheet_name=sheet, header=header_row)
                wb.close()

            # Обработка данных
            organizations = df[column_name].unique()
            total = len(organizations)
            os.makedirs(output_folder, exist_ok=True)
            
            for i, org in enumerate(organizations, 1):
                if not self.running:  # Проверка флага
                    break
                org_data = df[df[column_name] == org]
                clean_org_name = "".join([c if c.isalnum() or c in (' ', '_') else '' for c in str(org)])
                if org in vipnets:
                    output_path = os.path.join(output_folder, self.prefix.get()+' '+clean_org_name+vipnets[org]+'.xlsx')
                else:
                    print('нет правила для организации:', org)
                    output_path = os.path.join(output_folder, f"{clean_org_name}.xlsx")
                
                with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
                    org_data.to_excel(writer, index=False, sheet_name='Data')
                    if col_widths is not None:
                        worksheet = writer.sheets['Data']
                        for idx, width in enumerate(col_widths, 1):
                            worksheet.column_dimensions[get_column_letter(idx)].width = width
                progress = (i / total) * 100
                self.after(0, self.update_progress, progress)
                self.after(0, self.update_status, f"Обработано: {i} из {total} ({org})")
            
            if self.running:
                self.after(0, self.update_status, "Обработка завершена успешно!")
                self.after(0, lambda: messagebox.showinfo(
                    "Готово", 
                    f"Создано {total} файлов в папке:\n{output_folder}"
                ))
        
        except Exception as e:
            self.after(0, lambda: messagebox.showerror("Ошибка", str(e)))
            self.after(0, self.update_status, "Ошибка обработки!")
        finally:
            self.after(0, self.reset_progress)
            self.running = False
    
    def update_progress(self, value):
        self.progress_value.set(value)
    
    def reset_progress(self):
        self.progress_value.set(0)
    
    def update_status(self, message):
        self.status_text.set(message)
    
    def on_close(self):
        if messagebox.askokcancel("Выход", "Вы уверены, что хотите выйти?"):
            self.running = False  # Остановка фонового потока
            if hasattr(self, 'processing_thread') and self.processing_thread.is_alive():
                self.processing_thread.join(timeout=1)  # Ожидание завершения потока
            self.destroy()

if __name__ == "__main__":
    app = Application()
    app.mainloop()

In [1]:
import os
import csv
import threading
import openpyxl
import traceback
import re
from openpyxl.utils import get_column_letter
import pandas as pd
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from openpyxl.styles import NamedStyle
from datetime import datetime

class Application(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Фильтрация организаций")
        self.geometry("750x500")
        
        # Variables
        self.input_file = tk.StringVar()
        self.output_folder = tk.StringVar()
        self.column_name = tk.StringVar()
        self.sheet_name = tk.StringVar()
        self.header_row = tk.IntVar(value=1)
        self.status_text = tk.StringVar()
        self.progress_value = tk.DoubleVar()
        self.prefix = tk.StringVar()
        self.columns_list = []
        self.sheet_names = []
        self.preview_data = []
        self.vipnets_log = []
        
        # Flag for thread stopping
        self.running = False
        
        self.create_widgets()
        self.protocol("WM_DELETE_WINDOW", self.on_close)
    
    def create_widgets(self):
        input_frame = ttk.LabelFrame(self, text="Параметры обработки")
        input_frame.pack(padx=10, pady=5, fill='x')
        
        # File selection
        ttk.Label(input_frame, text="Входной файл:").grid(row=0, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.input_file, width=40).grid(row=0, column=1, padx=5, pady=2)
        ttk.Button(input_frame, text="Обзор...", command=self.browse_file).grid(row=0, column=2, padx=5, pady=2)
        
        # Sheet selection
        self.sheet_label = ttk.Label(input_frame, text="Лист в книге:")
        self.sheet_label.grid(row=1, column=0, sticky='w', padx=5, pady=2)
        self.sheet_combobox = ttk.Combobox(input_frame, textvariable=self.sheet_name, state="readonly", width=20)
        self.sheet_combobox.grid(row=1, column=1, sticky='w', padx=5, pady=2)
        
        # Header row selection
        ttk.Label(input_frame, text="Строка с заголовками:").grid(row=2, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.header_row, width=5).grid(row=2, column=1, sticky='w', padx=5, pady=2)
        ttk.Button(input_frame, text="Предпросмотр", command=self.show_preview).grid(row=2, column=2, padx=5, pady=2)
        
        # Column selection
        ttk.Label(input_frame, text="Столбец с организациями:").grid(row=3, column=0, sticky='w', padx=5, pady=2)
        self.column_combobox = ttk.Combobox(input_frame, textvariable=self.column_name, state="readonly", width=20)
        self.column_combobox.grid(row=3, column=1, sticky='w', padx=5, pady=2)
        
        # Output folder selection
        ttk.Label(input_frame, text="Папка для результатов:").grid(row=4, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.output_folder, width=40).grid(row=4, column=1, padx=5, pady=2)
        ttk.Button(input_frame, text="Обзор...", command=self.browse_output).grid(row=4, column=2, padx=5, pady=2)

        # File prefix
        ttk.Label(input_frame, text="Префикс для названия файлов:").grid(row=5, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.prefix, width=40).grid(row=5, column=1, padx=5, pady=2)
        self.prefix.set('Гепатиты 244-п август')
        
        # Progress and status
        progress_frame = ttk.Frame(self)
        progress_frame.pack(padx=10, pady=10, fill='x')
        self.progress = ttk.Progressbar(progress_frame, variable=self.progress_value, maximum=100)
        self.progress.pack(fill='x')
        
        status_frame = ttk.Frame(self)
        status_frame.pack(padx=10, pady=5, fill='x')
        ttk.Label(status_frame, textvariable=self.status_text).pack(side='left')
        
        # Buttons
        button_frame = ttk.Frame(self)
        button_frame.pack(padx=10, pady=10)
        ttk.Button(button_frame, text="Старт", command=self.start_processing).pack(side='left', padx=5)
        ttk.Button(button_frame, text="Выход", command=self.on_close).pack(side='left', padx=5)
    
    def browse_file(self):
        filetypes = (("Excel/CSV файлы", "*.xlsx *.xls *.csv"), ("Все файлы", "*.*"))
        filename = filedialog.askopenfilename(filetypes=filetypes)
        if filename:
            self.input_file.set(filename)
            self.load_file_metadata(filename)
    
    def load_file_metadata(self, filename):
        try:
            if filename.lower().endswith(('.xls', '.xlsx')):
                self.load_sheets(filename)
                self.load_columns()
            else:
                self.load_columns()
            
            self.show_preview()
            
        except Exception as e:
            messagebox.showerror("Ошибка", f"Ошибка загрузки файла: {str(e)}")
    
    def load_sheets(self, filename):
        try:
            wb = openpyxl.load_workbook(filename, read_only=True)
            self.sheet_names = wb.sheetnames
            wb.close()
            self.sheet_combobox['values'] = self.sheet_names
            self.sheet_name.set(self.sheet_names[0])
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось прочитать листы: {str(e)}")
    
    def load_columns(self):
        try:
            header_row = self.header_row.get() - 1  # Convert to 0-based index
            filename = self.input_file.get()
            
            if filename.endswith(('.xls', '.xlsx')):
                df = pd.read_excel(
                    filename,
                    sheet_name=self.sheet_name.get(),
                    header=header_row,
                    nrows=0
                )
            else:
                df = pd.read_csv(
                    filename,
                    header=header_row,
                    nrows=0,
                    encoding='utf-8'
                )
            
            self.columns_list = df.columns.tolist()
            self.column_combobox['values'] = self.columns_list
            if self.columns_list:
                self.column_name.set(self.columns_list[0])
            
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось прочитать заголовки: {str(e)}")
    
    def show_preview(self):
        try:
            filename = self.input_file.get()
            if not filename:
                return
            
            nrows = 15
            header_row = self.header_row.get() - 1
            
            if filename.endswith(('.xls', '.xlsx')):
                df = pd.read_excel(
                    filename,
                    sheet_name=self.sheet_name.get(),
                    header=None,
                    nrows=nrows
                )
            else:
                df = pd.read_csv(
                    filename,
                    header=None,
                    nrows=nrows,
                    encoding='utf-8'
                )
            
            preview_window = tk.Toplevel(self)
            preview_window.title("Предпросмотр данных")
            
            frame = ttk.Frame(preview_window)
            frame.pack(expand=True, fill='both', padx=5, pady=5)
            
            tree = ttk.Treeview(frame, columns=list(range(len(df.columns))), show="headings")
            vsb = ttk.Scrollbar(frame, orient="vertical", command=tree.yview)
            hsb = ttk.Scrollbar(frame, orient="horizontal", command=tree.xview)
            tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)
            
            tree.grid(row=0, column=0, sticky='nsew')
            vsb.grid(row=0, column=1, sticky='ns')
            hsb.grid(row=1, column=0, sticky='ew')
            frame.grid_columnconfigure(0, weight=1)
            frame.grid_rowconfigure(0, weight=1)
            
            for i in range(len(df.columns)):
                tree.heading(i, text=f"Col {i+1}")
                tree.column(i, width=100, anchor='w')
            
            for index, row in df.iterrows():
                tree.insert("", "end", values=tuple(row))
            
            if header_row < len(df):
                tree.tag_configure('header', background='#b3e6ff')
                children = tree.get_children()
                if header_row < len(children):
                    tree.item(children[header_row], tags=('header',))
            
            def on_double_click(event):
                item = tree.identify_row(event.y)
                column = tree.identify_column(event.x)
                
                if item and column:
                    children = tree.get_children()
                    row_index = children.index(item) + 1
                    col_index = int(column.replace("#", "")) - 1
                    
                    self.header_row.set(row_index)
                    self.load_columns()
                    
                    if self.columns_list and col_index < len(self.columns_list):
                        self.column_name.set(self.columns_list[col_index])
                    
                    preview_window.destroy()
            
            tree.bind("<Double-1>", on_double_click)
            
            button_frame = ttk.Frame(preview_window)
            button_frame.pack(fill='x', padx=5, pady=5)
            ttk.Button(button_frame, text="Закрыть", command=preview_window.destroy).pack(side='right')
            
            preview_window.geometry("800x400")
            
            instruction = ttk.Label(
                preview_window,
                text="Дважды щелкните по ячейке, чтобы выбрать строку заголовков и столбец для фильтрации",
                font=('Arial', 10)
            )
            instruction.pack(pady=5)
            
        except Exception as e:
            messagebox.showerror("Ошибка", f"Ошибка предпросмотра: {str(e)}")
    
    def browse_output(self):
        folder = filedialog.askdirectory()
        if folder:
            self.output_folder.set(folder)
    
    def start_processing(self):
        if not self.validate_input():
            return
        self.running = True
        self.processing_thread = threading.Thread(target=self.process_data)
        self.processing_thread.start()
    
    def validate_input(self):
        if not self.input_file.get():
            messagebox.showwarning("Ошибка", "Выберите входной файл!")
            return False
        if self.input_file.get().endswith(('.xls', '.xlsx')) and not self.sheet_name.get():
            messagebox.showwarning("Ошибка", "Выберите лист в книге!")
            return False
        if not self.column_name.get():
            messagebox.showwarning("Ошибка", "Выберите столбец с организациями!")
            return False
        return True

    def process_data(self):
        def safe_load_workbook(path):
            wb = openpyxl.load_workbook(path)
            # Repair style cache if needed
            if hasattr(wb, '_alignments') and not wb._alignments:
                wb._alignments = [openpyxl.styles.Alignment()]
            if hasattr(wb, '_fills') and not wb._fills:
                wb._fills = [openpyxl.styles.PatternFill()]
            if hasattr(wb, '_fonts') and not wb._fonts:
                wb._fonts = [openpyxl.styles.Font()]
            return wb
        def safe_save(workbook, path):
            # Ensure all style caches exist
            for style_list in ['_alignments', '_fills', '_fonts', '_borders', '_number_formats']:
                if not hasattr(workbook, style_list) or not getattr(workbook, style_list):
                    setattr(workbook, style_list, [None])  # Create empty style cache
            
            # Validate all cells
            for sheet in workbook:
                for row in sheet.iter_rows():
                    for cell in row:
                        if cell._style and not hasattr(cell._style, 'alignmentId'):
                            cell._style.alignmentId = 0
                        if cell._style and not hasattr(cell._style, 'fontId'):
                            cell._style.fontId = 0
            
            workbook.save(path)
        vipnets = {}
        with open('act_addr.txt') as in_file:
            csv_reader = csv.reader(in_file)
            for row in csv_reader:
                vipnets[row[0]]=row[1]
        try:
            # Initialize variables with clear names
            input_path = self.input_file.get()
            org_column_name = self.column_name.get()
            output_dir = self.output_folder.get() or 'output'
            sheet_name = self.sheet_name.get()
            header_row_num = self.header_row.get()  # 1-based
            
            # Validate inputs
            if not input_path:
                raise ValueError("No input file selected")
            if not os.path.exists(input_path):
                raise FileNotFoundError(f"Input file not found: {input_path}")
            
            # Load workbook with error context
            try:
                wb = safe_load_workbook(input_path)
                ws = wb[sheet_name] if sheet_name else wb.active
            except Exception as e:
                raise ValueError(f"Failed to load workbook: {str(e)}")
    
            # Find organization column with detailed error reporting
            org_col_idx = None
            try:
                header_row = ws[header_row_num]
                for idx, cell in enumerate(header_row, 1):  # 1-based
                    if cell.value == org_column_name:
                        org_col_idx = idx
                        break
                
                if org_col_idx is None:
                    available_columns = [cell.value for cell in header_row if cell.value]
                    raise ValueError(
                        f"Column '{org_column_name}' not found in header.\n"
                        f"Available columns: {available_columns}"
                    )
            except Exception as e:
                raise ValueError(f"Header processing failed: {str(e)}")
    
            # Collect organizations with bounds checking
            organizations = set()
            try:
                for row in ws.iter_rows(min_row=header_row_num + 1):
                    if len(row) < org_col_idx:
                        continue  # Skip rows that don't have our column
                    
                    org_cell = row[org_col_idx - 1]  # Convert to 0-based index
                    if org_cell.value:
                        organizations.add(str(org_cell.value))
            except Exception as e:
                raise ValueError(f"Error collecting organizations: {str(e)}")
    
            # Create output directory
            os.makedirs(output_dir, exist_ok=True)
    
            # Process each organization with progress tracking
            total_orgs = len(organizations)
            for i, org in enumerate(organizations, 1):
                if not self.running:
                    break
    
                # Create new workbook
                new_wb = openpyxl.Workbook()
                new_ws = new_wb.active
    
                # 1. Copy headers
                try:
                    header_row = ws[header_row_num]
                    for src_cell in header_row:
                        new_cell = new_ws.cell(
                            row=src_cell.row,
                            column=src_cell.column,
                            value=src_cell.value
                        )
                        new_cell._style = src_cell._style
                except Exception as e:
                    new_wb.close()
                    raise ValueError(f"Failed to copy headers: {str(e)}")
    
                # 2. Copy matching rows
                try:
                    for src_row in ws.iter_rows(min_row=header_row_num + 1):
                        if len(src_row) < org_col_idx:
                            continue
                            
                        if str(src_row[org_col_idx - 1].value) == org:
                            for src_cell in src_row:
                                new_cell = new_ws.cell(
                                    row=src_cell.row,
                                    column=src_cell.column,
                                    value=src_cell.value
                                )
                                new_cell._style = src_cell._style
                except Exception as e:
                    new_wb.close()
                    raise ValueError(f"Failed to copy data rows: {str(e)}")
                clean_org_name = "".join([c if c.isalnum() or c in (' ', '_') else '' for c in str(org)])
                if org in vipnets:
                    output_path = os.path.join(
                        output_dir, 
                        f"{self.prefix.get()} {clean_org_name}{vipnets[org]}.xlsx"
                    )
                    self.vipnets_log.append((org, vipnets[org]))
                else:
                    output_path = os.path.join(output_dir, f"{clean_org_name}.xlsx")
                    
                # Save the file
                try:
                    safe_save(new_wb, output_path)
                except Exception as e:
                    # Fallback to simple save if style repair fails
                    try:
                        new_wb.save(output_path)
                    except Exception as fallback_error:
                        # Last resort - save with pandas
                        pd.DataFrame(list(ws.values)).to_excel(output_path)
                        print(f"Used pandas fallback for {org} due to: {fallback_error}")
                
                new_wb.close()
    
                # Update progress
                progress = (i / total_orgs) * 100
                self.after(0, self.update_progress, progress)
                self.after(0, self.update_status, 
                          f"Processed {i}/{total_orgs}: {output_path}")
    
            messagebox.showinfo(
                "Complete",
                f"Successfully created {total_orgs} files in:\n{output_dir}"
            )

        except Exception as e:
            error_msg = f"ERROR:\n{str(e)}\n\n{traceback.format_exc()}"
            print(error_msg)
            messagebox.showerror("Processing Failed", "Failed to process file. Check console for details.\n", f"Last organization: {org if 'org' in locals() else 'N/A'}")
        finally:
            self.after(0, self.reset_progress)
            self.running = False
    
    def update_progress(self, value):
        self.progress_value.set(value)
    
    def reset_progress(self):
        self.progress_value.set(0)
    
    def update_status(self, message):
        self.status_text.set(message)
    
    def on_close(self):
        if messagebox.askokcancel("Выход", "Вы уверены, что хотите выйти?"):
            self.running = False  # Stop background thread
            if hasattr(self, 'processing_thread') and self.processing_thread.is_alive():
                self.processing_thread.join(timeout=1)  # Wait for thread to finish
            self.destroy()

if __name__ == "__main__":
    app = Application()
    app.mainloop()

Used pandas fallback for ГАУЗ СО "ГП4 г. Нижний Тагил" due to: <class 'openpyxl.styles.numbers.NumberFormat'>.formatCode should be <class 'str'> but value is <class 'NoneType'>
Used pandas fallback for ГАУЗ СО "ЦРБ г. Кушва" due to: <class 'openpyxl.styles.numbers.NumberFormat'>.formatCode should be <class 'str'> but value is <class 'NoneType'>
Used pandas fallback for ГАУЗ СО "Красноуральская ЦРБ" due to: <class 'openpyxl.styles.numbers.NumberFormat'>.formatCode should be <class 'str'> but value is <class 'NoneType'>
Used pandas fallback for ГАУЗ СО "ГБ № 4 г. Нижний Тагил" due to: <class 'openpyxl.styles.numbers.NumberFormat'>.formatCode should be <class 'str'> but value is <class 'NoneType'>
Used pandas fallback for ГАУЗ СО "Кировградская ЦРБ" due to: <class 'openpyxl.styles.numbers.NumberFormat'>.formatCode should be <class 'str'> but value is <class 'NoneType'>
Used pandas fallback for ФГБУЗ ЦМСЧ № 91 ФМБА России Лесной due to: <class 'openpyxl.styles.numbers.NumberFormat'>.format

In [2]:
import os
import csv
import threading
import openpyxl
import traceback
import re
from openpyxl.utils import get_column_letter
import pandas as pd
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from openpyxl.styles import NamedStyle
from datetime import datetime

class Application(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Фильтрация организаций")
        self.geometry("750x500")
        
        # Переменные
        self.input_file = tk.StringVar()
        self.output_folder = tk.StringVar()
        self.column_name = tk.StringVar()
        self.sheet_name = tk.StringVar()
        self.header_row = tk.IntVar(value=1)
        self.status_text = tk.StringVar()
        self.progress_value = tk.DoubleVar()
        self.prefix = tk.StringVar()
        self.columns_list = []
        self.sheet_names = []
        self.preview_data = []
        self.vipnets_log = []
        
        # Флаг остановки потока
        self.running = False
        
        self.create_widgets()
        self.protocol("WM_DELETE_WINDOW", self.on_close)
    
    def create_widgets(self):
        input_frame = ttk.LabelFrame(self, text="Параметры обработки")
        input_frame.pack(padx=10, pady=5, fill='x')
        
        # Выбор файла
        ttk.Label(input_frame, text="Входной файл:").grid(row=0, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.input_file, width=40).grid(row=0, column=1, padx=5, pady=2)
        ttk.Button(input_frame, text="Обзор...", command=self.browse_file).grid(row=0, column=2, padx=5, pady=2)
        
        # Выбор листа
        self.sheet_label = ttk.Label(input_frame, text="Лист в книге:")
        self.sheet_label.grid(row=1, column=0, sticky='w', padx=5, pady=2)
        self.sheet_combobox = ttk.Combobox(input_frame, textvariable=self.sheet_name, state="readonly", width=20)
        self.sheet_combobox.grid(row=1, column=1, sticky='w', padx=5, pady=2)
        
        # Строка с заголовком
        ttk.Label(input_frame, text="Строка с заголовками:").grid(row=2, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.header_row, width=5).grid(row=2, column=1, sticky='w', padx=5, pady=2)
        ttk.Button(input_frame, text="Предпросмотр", command=self.show_preview).grid(row=2, column=2, padx=5, pady=2)
        
        # Выбор столбца
        ttk.Label(input_frame, text="Столбец с организациями:").grid(row=3, column=0, sticky='w', padx=5, pady=2)
        self.column_combobox = ttk.Combobox(input_frame, textvariable=self.column_name, state="readonly", width=20)
        self.column_combobox.grid(row=3, column=1, sticky='w', padx=5, pady=2)
        
        # Выбор папки для вывода
        ttk.Label(input_frame, text="Папка для результатов:").grid(row=4, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.output_folder, width=40).grid(row=4, column=1, padx=5, pady=2)
        ttk.Button(input_frame, text="Обзор...", command=self.browse_output).grid(row=4, column=2, padx=5, pady=2)

        # Префикс для файлов
        ttk.Label(input_frame, text="Префикс для названия файлов:").grid(row=5, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.prefix, width=40).grid(row=5, column=1, padx=5, pady=2)
        self.prefix.set('Гепатиты 244-п август')
        
        # Прогрессбар и статус
        progress_frame = ttk.Frame(self)
        progress_frame.pack(padx=10, pady=10, fill='x')
        self.progress = ttk.Progressbar(progress_frame, variable=self.progress_value, maximum=100)
        self.progress.pack(fill='x')
        
        status_frame = ttk.Frame(self)
        status_frame.pack(padx=10, pady=5, fill='x')
        ttk.Label(status_frame, textvariable=self.status_text).pack(side='left')
        
        # Кнопки
        button_frame = ttk.Frame(self)
        button_frame.pack(padx=10, pady=10)
        ttk.Button(button_frame, text="Старт", command=self.start_processing).pack(side='left', padx=5)
        ttk.Button(button_frame, text="Выход", command=self.on_close).pack(side='left', padx=5)
    
    def browse_file(self):
        filetypes = (("Excel/CSV файлы", "*.xlsx *.xls *.csv"), ("Все файлы", "*.*"))
        filename = filedialog.askopenfilename(filetypes=filetypes)
        if filename:
            self.input_file.set(filename)
            self.load_file_metadata(filename)
    
    def load_file_metadata(self, filename):
        try:
            if filename.lower().endswith(('.xls', '.xlsx')):
                self.load_sheets(filename)
                self.load_columns()
            else:
                self.load_columns()
            
            self.show_preview()
            
        except Exception as e:
            messagebox.showerror("Ошибка", f"Ошибка загрузки файла: {str(e)}")
    
    def load_sheets(self, filename):
        try:
            wb = openpyxl.load_workbook(filename, read_only=True)
            self.sheet_names = wb.sheetnames
            wb.close()
            self.sheet_combobox['values'] = self.sheet_names
            self.sheet_name.set(self.sheet_names[0])
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось прочитать листы: {str(e)}")
    
    def load_columns(self):
        try:
            header_row = self.header_row.get() - 1  # Приводим к 0-индексации
            filename = self.input_file.get()
            
            if filename.endswith(('.xls', '.xlsx')):
                df = pd.read_excel(
                    filename,
                    sheet_name=self.sheet_name.get(),
                    header=header_row,
                    nrows=0
                )
            else:
                df = pd.read_csv(
                    filename,
                    header=header_row,
                    nrows=0,
                    encoding='utf-8'
                )
            
            self.columns_list = df.columns.tolist()
            self.column_combobox['values'] = self.columns_list
            if self.columns_list:
                self.column_name.set(self.columns_list[0])
            
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось прочитать заголовки: {str(e)}")
    
    def show_preview(self):
        try:
            filename = self.input_file.get()
            if not filename:
                return
            
            nrows = 15
            header_row = self.header_row.get() - 1
            
            if filename.endswith(('.xls', '.xlsx')):
                df = pd.read_excel(
                    filename,
                    sheet_name=self.sheet_name.get(),
                    header=None,
                    nrows=nrows
                )
            else:
                df = pd.read_csv(
                    filename,
                    header=None,
                    nrows=nrows,
                    encoding='utf-8'
                )
            
            preview_window = tk.Toplevel(self)
            preview_window.title("Предпросмотр данных")
            
            frame = ttk.Frame(preview_window)
            frame.pack(expand=True, fill='both', padx=5, pady=5)
            
            tree = ttk.Treeview(frame, columns=list(range(len(df.columns))), show="headings")
            vsb = ttk.Scrollbar(frame, orient="vertical", command=tree.yview)
            hsb = ttk.Scrollbar(frame, orient="horizontal", command=tree.xview)
            tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)
            
            tree.grid(row=0, column=0, sticky='nsew')
            vsb.grid(row=0, column=1, sticky='ns')
            hsb.grid(row=1, column=0, sticky='ew')
            frame.grid_columnconfigure(0, weight=1)
            frame.grid_rowconfigure(0, weight=1)
            
            for i in range(len(df.columns)):
                tree.heading(i, text=f"Col {i+1}")
                tree.column(i, width=100, anchor='w')
            
            for index, row in df.iterrows():
                tree.insert("", "end", values=tuple(row))
            
            if header_row < len(df):
                tree.tag_configure('header', background='#b3e6ff')
                children = tree.get_children()
                if header_row < len(children):
                    tree.item(children[header_row], tags=('header',))
            
            def on_double_click(event):
                item = tree.identify_row(event.y)
                column = tree.identify_column(event.x)
                
                if item and column:
                    children = tree.get_children()
                    row_index = children.index(item) + 1
                    col_index = int(column.replace("#", "")) - 1
                    
                    self.header_row.set(row_index)
                    self.load_columns()
                    
                    if self.columns_list and col_index < len(self.columns_list):
                        self.column_name.set(self.columns_list[col_index])
                    
                    preview_window.destroy()
            
            tree.bind("<Double-1>", on_double_click)
            
            button_frame = ttk.Frame(preview_window)
            button_frame.pack(fill='x', padx=5, pady=5)
            ttk.Button(button_frame, text="Закрыть", command=preview_window.destroy).pack(side='right')
            
            preview_window.geometry("800x400")
            
            instruction = ttk.Label(
                preview_window,
                text="Дважды щелкните по ячейке, чтобы выбрать строку заголовков и столбец для фильтрации",
                font=('Arial', 10)
            )
            instruction.pack(pady=5)
            
        except Exception as e:
            messagebox.showerror("Ошибка", f"Ошибка предпросмотра: {str(e)}")
    
    def browse_output(self):
        folder = filedialog.askdirectory()
        if folder:
            self.output_folder.set(folder)
    
    def start_processing(self):
        if not self.validate_input():
            return
        self.running = True
        self.processing_thread = threading.Thread(target=self.process_data)
        self.processing_thread.start()
    
    def validate_input(self):
        if not self.input_file.get():
            messagebox.showwarning("Ошибка", "Выберите входной файл!")
            return False
        if self.input_file.get().endswith(('.xls', '.xlsx')) and not self.sheet_name.get():
            messagebox.showwarning("Ошибка", "Выберите лист в книге!")
            return False
        if not self.column_name.get():
            messagebox.showwarning("Ошибка", "Выберите столбец с организациями!")
            return False
        return True


    def process_data(self):
        vipnets = {}
        with open('act_addr.txt') as in_file:
            csv_reader = csv.reader(in_file)
            for row in csv_reader:
                vipnets[row[0]]=row[1]
        
        try:
            input_path = self.input_file.get()
            org_column_name = self.column_name.get()
            output_dir = self.output_folder.get() or 'output'
            sheet_name = self.sheet_name.get()
            header_row_num = self.header_row.get()  # 1-based
            
            # Читаем заголовки первой строки
            df = pd.read_excel(input_path, sheet_name=sheet_name, header=header_row_num - 1, nrows=0)
            headers = df.columns.tolist()
            
            # Находим индекс колонки
            org_col_idx = None
            for idx, col_header in enumerate(headers, 1):
                if col_header == org_column_name:
                    org_col_idx = idx
                    break
            
            if org_col_idx is None:
                raise ValueError(f"Столбец '{org_column_name}' не найден.")
            
            # Загрузка рабочей книги
            wb = openpyxl.load_workbook(input_path)
            ws = wb[sheet_name] if sheet_name else wb.active
            
            # Собираем уникальные организации
            organizations = set()
            for row in ws.iter_rows(min_row=header_row_num + 1):
                if len(row) >= org_col_idx:
                    org = row[org_col_idx - 1].value
                    if org:
                        organizations.add(org)
            
            # Обрабатываем каждую организацию
            total_orgs = len(organizations)
            for i, org in enumerate(sorted(organizations), 1):
                if not self.running:
                    break
                
                # Удаляем ненужные строки
                rows_to_delete = []
                for row in ws.iter_rows(min_row=header_row_num + 1):
                    if len(row) >= org_col_idx and row[org_col_idx - 1].value != org:
                        rows_to_delete.append(row[0].row)
                
                # Массив для удаления сразу всех строк
                for row_idx in sorted(rows_to_delete, reverse=True):
                    ws.delete_rows(row_idx)
                
                # Очищаем имя организации для файла
                clean_org_name = ''.join(c if c.isalnum() or c in (' ', '_') else '_' for c in str(org))
                
                # Формирование пути к файлу
                if org in vipnets:
                    output_path = os.path.join(output_dir, f"{self.prefix.get()} {clean_org_name}{vipnets[org]}.xlsx")
                    self.vipnets_log.append((org, vipnets[org]))
                else:
                    output_path = os.path.join(output_dir, f"{clean_org_name}.xlsx")
                
                # Сохраняем новый файл
                wb.save(output_path)
                
                # Восстанавливаем исходную книгу
                wb = openpyxl.load_workbook(input_path)
                ws = wb[sheet_name] if sheet_name else wb.active
                
                # Показываем прогресс
                progress = (i / total_orgs) * 100
                self.after(0, self.update_progress, progress)
                self.after(0, self.update_status, f"Обработано {i}/{total_orgs}: {output_path}")
            
            messagebox.showinfo("Готово", f"Успешно обработаны {len(organizations)} файлов.")
        
        except Exception as e:
            error_msg = f"Ошибка обработки: {str(e)}"
            print(traceback.format_exc())
            messagebox.showerror("Ошибка", error_msg)
        finally:
            self.after(0, self.reset_progress)
            self.running = False

    def update_progress(self, value):
        self.progress_value.set(value)
    
    def reset_progress(self):
        self.progress_value.set(0)
    
    def update_status(self, message):
        self.status_text.set(message)
    
    def on_close(self):
        if messagebox.askokcancel("Выход", "Вы действительно хотите закрыть программу?"):
            self.running = False
            if hasattr(self, 'processing_thread') and self.processing_thread.is_alive():
                self.processing_thread.join(timeout=1)
            self.destroy()

if __name__ == "__main__":
    app = Application()
    app.mainloop()